In [3]:
import duckdb
import pandas as pd
import os

# Connect to a persistent DuckDB database file
conn = duckdb.connect("imdb.duckdb")

# Create tables and load data
def create_imdb_tables():
    # Title AKAs
    conn.execute(f"""
    CREATE OR REPLACE TABLE title_akas AS
    SELECT 
        titleId AS title_id,
        ordering::INTEGER,
        title,
        region,
        language,
        string_split(types, ',') AS types,
        string_split(attributes, ',') AS attributes,
        isOriginalTitle::BOOLEAN AS is_original_title
    FROM read_csv('{os.path.join('data','title.akas.tsv')}', 
        delim='\t', 
        header=True, 
        nullstr='\\N')
    """)
    
    # Title Basics
    conn.execute(f"""
    CREATE OR REPLACE TABLE title_basics AS
    SELECT 
        tconst,
        titleType AS title_type,
        primaryTitle AS primary_title,
        originalTitle AS original_title,
        TRY_CAST(isAdult AS BOOLEAN) AS is_adult,
        TRY_CAST(startYear AS INTEGER) AS start_year,
        TRY_CAST(endYear AS INTEGER) AS end_year,
        TRY_CAST(runtimeMinutes AS INTEGER) AS runtime_minutes,
        string_split(genres, ',') AS genres
    FROM read_csv('{os.path.join('data','title.basics.tsv')}', 
        delim='\t', 
        header=True, 
        nullstr='\\N')
    """)
    
    # Title Crew
    conn.execute(f"""
    CREATE OR REPLACE TABLE title_crew AS
    SELECT 
        tconst,
        string_split(directors, ',') AS directors,
        string_split(writers, ',') AS writers
    FROM read_csv('{os.path.join('data','title.crew.tsv')}', 
        delim='\t', 
        header=True, 
        nullstr='\\N')
    """)
    
    # Title Episode
    conn.execute(f"""
    CREATE OR REPLACE TABLE title_episode AS
    SELECT 
        tconst,
        parentTconst AS parent_tconst,
        TRY_CAST(seasonNumber AS INTEGER) AS season_number,
        TRY_CAST(episodeNumber AS INTEGER) AS episode_number
    FROM read_csv('{os.path.join('data','title.episode.tsv')}', 
        delim='\t', 
        header=True, 
        nullstr='\\N')
    """)
    
    # Title Principals
    conn.execute(f"""
    CREATE OR REPLACE TABLE title_principals AS
    SELECT 
        tconst,
        ordering::INTEGER,
        nconst,
        category,
        job,
        characters
    FROM read_csv('{os.path.join('data','title.principals.tsv')}', 
        delim='\t', 
        header=True, 
        nullstr='\\N')
    """)
    
    # Title Ratings
    conn.execute(f"""
    CREATE OR REPLACE TABLE title_ratings AS
    SELECT 
        tconst,
        averageRating::FLOAT AS average_rating,
        numVotes::INTEGER AS num_votes
    FROM read_csv('{os.path.join('data','title.ratings.tsv')}', 
        delim='\t', 
        header=True, 
        nullstr='\\N')
    """)
    
    # Name Basics
    conn.execute(f"""
    CREATE OR REPLACE TABLE name_basics AS
    SELECT 
        nconst,
        primaryName AS primary_name,
        TRY_CAST(birthYear AS INTEGER) AS birth_year,
        TRY_CAST(deathYear AS INTEGER) AS death_year,
        string_split(primaryProfession, ',') AS primary_profession,
        string_split(knownForTitles, ',') AS known_for_titles
    FROM read_csv('{os.path.join('data','name.basics.tsv')}', 
        delim='\t', 
        header=True, 
        nullstr='\\N')
    """)

    print("All IMDb tables created successfully!")

# Execute table creation
create_imdb_tables()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

All IMDb tables created successfully!


Create optimized Parquet versions.

In [4]:
conn.execute("""
    EXPORT DATABASE 'parquet' (
        FORMAT PARQUET,
        COMPRESSION ZSTD
    )
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Now we have created 7 tables:
1. title_akas
2. title_basics
3. title_crew
4. title_episode
5. title_principals
6. title_ratings
7. name_basics

Key Features:

- Uses DuckDB's native CSV reader with proper delimiters and null handling

- Converts comma-separated strings to DuckDB LIST types

- Handles type casting with TRY_CAST for safety

- Creates persistent DuckDB database file

- Optional Parquet export for faster queries

Usage:

In [6]:
df = conn.execute("""
    SELECT 
        *
    FROM title_basics tb
    JOIN title_ratings tr 
        ON tb.tconst = tr.tconst
    WHERE tb.title_type = 'movie'
        AND tb.start_year >= 2020
    ORDER BY tr.average_rating DESC
    LIMIT 10
""").df()

display(df)

,tconst,title_type,primary_title,original_title,is_adult,start_year,end_year,runtime_minutes,genres,tconst_1,average_rating,num_votes
0,tt34695998,movie,Nenapugala Maathu Madhura,Nenapugala Maathu Madhura,False,2025,<NA>,<NA>,<NA>,tt34695998,10.0,208
1,tt33839333,movie,The Cry of O Neill,The Cry of O Neill,False,2024,<NA>,7,<NA>,tt33839333,10.0,12
2,tt19497458,movie,America Gone Viral,America Gone Viral,False,2022,<NA>,84,[Drama],tt19497458,10.0,9
3,tt17717882,movie,Annie Johnson's Brothers,Annie Johnson's Brothers,False,2021,<NA>,120,[Drama],tt17717882,10.0,11
4,tt14546442,movie,Pandemania,Pandemania,False,2021,<NA>,63,[Documentary],tt14546442,10.0,8
5,tt34754032,movie,El Vocho del Averno,El Vocho del Averno,False,2025,<NA>,<NA>,"[Comedy, Horror]",tt34754032,10.0,14
6,tt20770330,movie,The Roping Fools,The Roping Fools,False,2025,<NA>,<NA>,[Documentary],tt20770330,10.0,11
7,tt35665272,movie,Cuando lo peor haya pasado,Cuando lo peor haya pasado,False,2023,<NA>,<NA>,[Drama],tt35665272,10.0,6
8,tt29259789,movie,Sisters & The Shrink 4,Sisters & The Shrink 4,False,2023,<NA>,<NA>,[Drama],tt29259789,10.0,11
9,tt28420272,movie,Hora Uncle,Hora Uncle,False,2024,<NA>,<NA>,[Drama],tt28420272,10.0,17


A few tips to improve performance:
1. Use `WHERE` clause to filter early.
2. Create indexes on frequently used columns
```conn.execute("CREATE INDEX idx_tconst ON title_basics(tconst)")```
3. Use DuckDB's parallel processing:
```conn.execute("SET threads TO 8")```

Once you have finished, please close the connection. Otherwise you cannot use the database in other notebooks.

In [7]:
conn.close()